In [ ]:
use_dates = ["2016-01-28","2016-02-28","2016-03-28","2016-04-28","2016-05-28"]

trn = df_trn[df_trn["fecha_dato"].isin(use_dates)]
tst = df_trn[df_trn["fecha_dato"] == "2016-06-28"]

del df_trn

X, Y = [], []

for i, prod in enumerate(prods):
  prev = prod + "_prev"
  prX = trn[(trn[prod]==1)&(trn[prod]==0)]
  prY = np.zeros(prX.shape[0],dtype=np.int8) + i
  X.append(prX)
  Y.append(prY)

XY = pd.concat(X)
Y = np.hstack(Y)
XY['y'] = Y

vld_date = "2016-05-28"
XY_trn = XY[XY['fecha_dato'] != vld_date]
XY_vld = XY[XY['fecha_dato'] == vld_date]

In [ ]:
import xgboost as xgb

param = {
    "booster":"gbtree",
    "max_depth":8,
    "nthread":4,
    "num_class":len(prods),
    "objective":"multi:softprob",
    "silent":1,
    "eval_metric":"mlogloss",
    "eta":0.1,
    "min_child_weight":10,
    "colsample_bytree":0.8,
    "colsample_bylevel":0.9,
    "seed":2021,
}

# mapk
def apk(actual, predicted, k=7, default=0.0):
  if len(predicted) > k:
    predicted = predicted[:k]

  score = 0.0
  num_hits = 0.0

  for i,p in enumerate(predicted):
    if p in actual and p not in predicted[:i]:
      num_hits += 1.0
      score += num_hits/(i+1.0)

  if not actual:
    return default

  return score / min(len(actual),k)

def mapk(actual, predicted, k=7, default=0.0):
  return np.mean([apk(a,p,k,default) for a,p in zip(actual,predicted)])

# XGBoost 형태에 맞게 데이터 변형
X_trn = XY_trn.as_matrix(columns=features)
Y_trn = XY_trn.as_matrix(columns=['y'])
dtrn = xgb.DMatrix(X_trn, label=Y_trn, feature_names=features)

X_vld = XY_vld.as_matrix(columns=features)
Y_vld = XY_vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)

# 모델 정의 및 학습
watch_list = [(dtrn,"train"),(dvld,'eval')]
model = xgb.train(param,dtrn,num_boost_round=1000,evals=watch_list,early_stopping_rounds=20)

# 모델 저장
import pickle
pickle.dump(model,open("model/xgb.baseline.pkl","wb"))
base_ntree_limit = model.best_ntree_limit

# 검증을 위한 MAP@7
vld = trn[trn["fecha_dato"] == vld_date]
ncodpers_vld = vld.as_matrix(columns=['ncodpers'])

for prod in prods:
  pre = prod + "_prev"
  padd = prod + "_add"
  vld[padd] = vld[prod] - vld[prev]
add_vld = vld.as_matrix(columns=[prod + "_add" for prod in prods])
add_vld_list = [list() for i in range(len(ncodpers_vld))]

count_vld = 0
for ncodper in range(len(ncodpers_vld)):
  for prod in range(len(prods)):
    if add_vld[ncodper,prod] > 0:
      add_vld_list[ncodper].append(prod)
      count_vlt += 1

X_vld = vld.as_matrix(columns=features)
Y_vld = vld.as_matrix(columns=['y'])
dvld = xgb.DMatrix(X_vld, label=Y_vld, feature_names=features)
preds_vld = model.predict(dvld, ntree_limit=best_ntree_limit)

preds_vld = preds_vld - vld.as_matrix(columns=[prod+"_prev" for prod in prods])

result_vld = []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
  y_prods = [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
  y_prods = sorted(y_prods, key=lambda x: x[0], reverse=True)[:7]
  result_vld.append([ip for y,p,ip in y_prods])

print(mapk(add_vold_list,result_vld,7,0.0))

